# get prices to db

##IMPORTANT :     DO NOT RUN this notebook in Google Colab as there there is a permission issue with the API in this environment (error 451). Run it on the local machine, instead.

# Modules

In [ ]:
!pip install pycoingecko

In [ ]:
!pip install sqlalchemy

In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime

import warnings
from sqlalchemy import create_engine
import sqlite3
import requests
from tqdm import tqdm
from google.colab import drive
import os


In [ ]:
# Create a SQLite database connection
engine = create_engine('sqlite:///binance_prices.db')


# Get the list of coins quoted in USDT from Binance

In [ ]:
def get_current_traded_coins_in_usdt():
    """
    Fetch the current traded coin pairs in USDT from Binance API.
    DO NOT RUN the script in Google Colab as there there is a permission issue in the API
    in this environment (error 451)
    Run it on the local machine.

    Parameters:
    ----------
    None

    Returns:
    -------
    list
        A list of tuples containing the symbol, base asset, and quote asset of each traded pair.
    """
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)

    # Check for a successful response
    if response.status_code == 200:
        data = response.json()
        symbols = []
        for symbol in data['symbols']:
            if symbol['quoteAsset'] == 'USDT':
                symbols.append((symbol['symbol'],symbol['baseAsset'],symbol['quoteAsset']))

        print(f"Total pairs fetched: {len(symbols)}")
        return list(set(symbols))
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return []

current_pairs = get_current_traded_coins_in_usdt()
len(current_pairs)
current_pairs = pd.DataFrame(current_pairs, columns = ['symbol','base_currency','quote_currency'])

Failed to fetch data: 451


In [ ]:
current_pairs.head()

,symbol,base_currency,quote_currency


# remove stable coins

In [ ]:
# Function to check if a coin is likely a stablecoin
def is_stablecoin(base_currency):
    """
    Check if a given currency is a stablecoin.

    Parameters:
    ----------
    base_currency : str
        The currency code to check.

    Returns:
    -------
    bool
        True if the currency is a stablecoin, False otherwise.
    """
    stablecoin_identifiers = ['USD', 'USDT', 'DAI', 'USDC', 'PAX', 'TUSD', 'BUSD', 'GUSD']
    return any(coin in base_currency for coin in stablecoin_identifiers)

# Filter out stablecoins
non_stablecoins_list = current_pairs[~current_pairs.apply(lambda x: is_stablecoin(x['base_currency']), axis=1)]

non_stablecoins_list.shape

# Get historical data from Binance and save to database

In [ ]:
def get_binance_data(symbol, interval, start_str, end_str):
    """
    Fetch historical data from Binance API with pagination.

    Parameters:
    ----------
    symbol : str
        The trading pair symbol
    interval : str
        The interval for the kline/candlestick data .
    start_str : str
        The start date in 'YYYY-MM-DD' format.
    end_str : str
        The end date in 'YYYY-MM-DD' format.

    Returns:
    -------
    list
        A list of lists containing the historical data.
    """
    url = "https://api.binance.com/api/v3/klines"
    start_time = int(datetime.strptime(start_str, '%Y-%m-%d').timestamp() * 1000)  # Convert to milliseconds
    end_time = int(datetime.strptime(end_str, '%Y-%m-%d').timestamp() * 1000)  # Convert to milliseconds

    # Container for all the data
    all_data = []

    while start_time < end_time:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'limit': 1000
        }
        response = requests.get(url, params=params)
        data = response.json()
        if not data:
            break
        all_data.extend(data)
        last_entry = data[-1][0]  # Get the open time of the last entry
        start_time = last_entry + (60 * 1000)  # Add one min in milliseconds

    return all_data

def convert_to_dataframe(data):
    """
    Convert API response data into a pandas DataFrame.

    Parameters:
    ----------
    data : list
        A list of lists containing the historical data.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the historical data.
    """
    df = pd.DataFrame(data, columns=['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time',
                                     'Quote Asset Volume', 'Number of Trades', 'Taker buy base asset volume',
                                     'Taker buy quote asset volume', 'Ignore'])
    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
    df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
    df['Open'] = df['Open'].astype('float64')
    df['High'] = df['High'].astype('float64')
    df['Low'] = df['Low'].astype('float64')
    df['Close'] = df['Close'].astype('float64')
    df['Mid'] = (df['High']+df['Low'])/2
    df['Volume'] = df['Volume'].astype('float64')
    df['Quote Asset Volume'] = df['Quote Asset Volume'].astype('float64')
    df['Taker buy base asset volume'] = df['Taker buy base asset volume'].astype('float64')
    df['Taker buy quote asset volume'] = df['Taker buy quote asset volume'].astype('float64')
    df.drop(columns = ['Ignore'], inplace = True)
    df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
       'taker_buy_quote_asset_volume', 'mid']
    return df





In [ ]:
# Parameters
interval = '1m'
start_str = '2023-01-01'
end_str = '2024-01-31'

# Fetch data
with tqdm(total = non_stablecoins_list.shape[0], desc = 'processing pairs', bar_format = '{l_bar}{bar} | {n_fmt}/{total_fmt}') as pbar:
    for _,pair in non_stablecoins_list.iterrows():
        data = get_binance_data(pair['symbol'], interval, start_str, end_str)
        df = convert_to_dataframe(data).sort_values(by = 'close_time', ascending = True)
        df['symbol'] = pair['symbol']
        df['base_currency'] = pair['base_currency']
        df['quote_currency'] = pair['quote_currency']
        df['monetary_volume'] = df['mid'].fillna(0) * df['volume'].fillna(0)
        # Export the DataFrame to the SQLite database
        df.to_sql('market_data', con=engine, index=False, if_exists='append')

        pbar.update(df.shape[0])